# Absolute GLAM Fit
This notebook estimates parameters for the GLAM to half of the data and then simulates the held out trials. For quantification of the absolute fit, see the _GLAM_OOS_ModelFit.mlx_ analysis script in this repository.

## Housekeeping
### Load Necessary Libraries

In [1]:
import warnings
from os.path import join

import glambox as gb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Ignore future warnings re: multi-indexing
warnings.simplefilter(action="ignore", category=FutureWarning)

# Fix for compilation error (as per Felix Molter)
# https://stackoverflow.com/a/51312739
import theano

theano.config.gcc.cxxflags = "-Wno-c++11-narrowing"
theano.config.cxx


'/usr/bin/clang++'

### Set MCMC Sampling Parameters

In [2]:
# Set MCMC parameters
n_tune = 5000
n_draws = 5000
n_chains = 2

### Set Other Miscellaneous Variables

In [3]:
# Set error weight
error_weight = 0.01

#
# set monkey ID
monkeys = ["MonkeyC", "MonkeyK"]

### Set Data Grouping Flag
This notebook can be run treating each monkey as a subject, or each session as a subject. The data for the two versions are are in separate excel files. The only difference between the two is the input to the 'subject' field. 

**To select the correct input file, set the variable DATA_GROUPING to either 'session' or 'monkey'.** 'monkey' is the default option

In [4]:
DATA_GROUPING = "monkey"

In [5]:
data = pd.read_csv("GLAMdata_"+ DATA_GROUPING + ".csv")

# display the unique subject numbers to make sure you set the desired data grouping flag
print(data.subject.unique())

[0 1]


#### Split data into even and odd trials

In [6]:
data_odd = pd.DataFrame()
data_even = pd.DataFrame()
subIds = data.subject.unique()
for s in subIds:
    tmp_odd = data.loc[(data["trial"] % 2 == 1) & (data["subject"] == s)]
    tmp_even = data.loc[(data["trial"] % 2 == 0) & (data["subject"] == s)]
    data_even = data_even.append(tmp_even)
    data_odd = data_odd.append(tmp_odd)

## Parameter Estimation
This section uses the even-numbered trials to estimate the GLAM parameters.

In [7]:
# initialize GLAM model object with the even-numbered trials
model = gb.GLAM(data=data_even)

In [8]:
# define model. 
model.make_model(kind="individual", t0_val=0, error_weight=error_weight)

Generating single subject models for 2 subjects...
Elemwise{add,no_inplace}.0
Elemwise{add,no_inplace}.0
Elemwise{add,no_inplace}.0
Elemwise{add,no_inplace}.0
Elemwise{add,no_inplace}.0
Elemwise{add,no_inplace}.0


In [9]:
# fit model to the data
model.fit(tune=n_tune, draws=n_draws, chains=n_chains)

Fitting 2 model(s) using MCMC...
  Fitting model 1 of 2...


Multiprocess sampling (2 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 2 chains: 100%|██████████████| 20000/20000 [01:37<00:00, 205.05draws/s]
The number of effective samples is smaller than 10% for some parameters.


  Fitting model 2 of 2...


Multiprocess sampling (2 chains in 4 jobs)
CompoundStep
>Metropolis: [tau]
>Metropolis: [s]
>Metropolis: [gamma]
>Metropolis: [v]
Sampling 2 chains: 100%|██████████████| 20000/20000 [01:30<00:00, 221.00draws/s]
The number of effective samples is smaller than 10% for some parameters.


/!\ Automatically setting parameter precision...


In [10]:
# Show parameter estimates
# Monkey C is row 0 and Monkey K is row 1.
model.estimates

,gamma,gamma_hpd_2.5,gamma_hpd_97.5,s,s_hpd_2.5,s_hpd_97.5,subject,t0,t0_hpd_2.5,t0_hpd_97.5,tau,tau_hpd_2.5,tau_hpd_97.5,v,v_hpd_2.5,v_hpd_97.5
0,0.08,0.047285,0.11977,0.196,0.192877,0.199230,0,0.0,0.0,0.0,0.263,0.254325,0.270821,2.312,2.301951,2.325475
1,-0.05,-0.083737,-0.01550,0.171,0.168130,0.173755,1,0.0,0.0,0.0,0.303,0.294645,0.312501,2.205,2.192398,2.216070


## Out-of-Sample Simulations
This section uses the estimated parameters to simulate each of the held-out (odd-numbered) trials. See 'GLAM_OOS_ModelFit.mlx' in the _Matlab_ folder of this repository for visualization and assessment of the out-of-sample simulations' goodness-of-fit. 

In [11]:
# swap out the even-numbered trials with the held-out odd trials. 
model.exchange_data(new_data=data_odd)

Replaced attached data (15024 trials) with new data (15022 trials)...


In [12]:
# simulate each of the held-out trials 10x
model.predict(n_repeats=10, error_weight=error_weight)

  0%|                                         | 3/15022 [00:00<10:24, 24.06it/s]

Generating predictions for 15022 trials (10 repeats each)...


100%|█████████████████████████████████████| 15022/15022 [23:46<00:00, 10.53it/s]


### Visualize Results
The visualization shown in the manuscript was generated using the _GLAM_OOS_ModelFit.mlx_ file. The mat file loaded by that script uses data exported from python 